The set $V$ can be computed by initially setting $V = \emptyset$. Add to $V$ the neighbours of $u$, then find the vertices of $X$ matched to $V$, then add to $V$ the neighbours of those vertices, and so on. For each vertex added, the preceding vertex should be recorded which led to the addition.

Given a bipartite graph $G$ with vertex classes $X$ and $Y$ of equal size $n$, find a $1$-factor (a matching of size $n$) or determine that one does not exist.

Main loop:

1.  Initialization Start with an empty matching, $M = $∅.
2.  If $|M| = n$, then terminate and output $M$ as the $1$-factor.
3.  Otherwise, while $|M$ < n$:

    *   Select an unmatched vertex $u$ from the vertex class $X$.
    *   Begin a search for an augmenting path starting from $u$. This search will identify all vertices reachable from $u$ via an alternating path.

Augmenting path search (from $u$):

1.  Create a set of "reachable" vertices, initialized with $u$. Maintain a record of the path (predecessor pointers) for each vertex that is discovered.
2.  Explore the graph in layers, alternating between unmatched and matched edges:

    *   Layer 1 ($X \rightarrow Y$): Find all neighbors of $u \in Y$ that are connected by an edge not in $M$.
    *   Layer 2 ($Y \rightarrow X$): For each new vertex $y$ found in the previous layer, find the vertex $x \in X$ that is matched to $y$ by an edge in $M$.
    *   Subsequent layers: Repeat this process, finding neighbors via edges not in $M$ (from $X$ to $Y$) and then partners via edges in $M$ (from $Y$ to $X$), until no new vertices can be reached.

3.  Throughout this search, if a vertex $v$ in $Y$ is reached that is unmatched in $M$, then an augmenting path has been found. Proceed to augment matching.
4.  If the search completes and all reachable vertices have been explored without finding an unmatched vertex in $Y$, then no augmenting path exists from $u$. Terminate and output the blocking set of reachable vertices in $X$

Augment matching:
1.  If an augmenting path $P$ is found (from the unmatched vertex $u \in X$ to an unmatched vertex $v \in Y$), then use the recorded predecessor information to reconstruct this path.
2.  Create a new, larger matching $M'$ by taking the symmetric difference of the current matching $M$ and the edges of the path $P$:
    *   Removing from $M$ all edges that are in both $M$ and $P$.
    *   Adding to $M$ all edges from $P$ that were not previously in $M$.
3.  Replace the current matching $M$ with the new matching $M'$. The size of the matching has now increased by $1$.
4.  Return to the main loop to find the next augmenting path.

In [ ]:
import numpy as np
import math
import random
from collections import deque

class BipartiteGraph:
    '''
    Represents a bipartite graph with vertex sets X and Y, each of size n.
    The graph can be generated randomly or built incrementally.
    '''
    def __init__(self, n, p=0):
        '''
        Initializes the graph.
        - n: Number of vertices in each partition (X and Y).
        - p: Edge probability for random graph generation. If 0, an empty graph is made.
        '''
        self.n = n
        # Adjacency matrix: adj[u][v] = 1 if edge (u_in_X, v_in_Y) exists.
        if p > 0:
            self.adj = (np.random.rand(n, n) < p).astype(int)
        else:
            self.adj = np.zeros((n, n), dtype=int)

    def add_edge(self, u, v):
        '''
        Adds an edge from u in X to v in Y.
        '''
        self.adj[u, v] = 1

    def get_neighbors(self, u_in_X):
        '''
        Returns a list of neighbors in Y for a vertex u in X.
        '''
        return [v for v in range(self.n) if self.adj[u_in_X, v]]

    def has_isolated_vertices(self):
        '''
        Checks if there is any vertex with degree 0.
        '''
        # Check if any row sum (degrees of X vertices) is zero
        if np.any(np.sum(self.adj, axis=1) == 0):
            return True
        # Check if any column sum (degrees of Y vertices) is zero
        if np.any(np.sum(self.adj, axis=0) == 0):
            return True
        return False

In [ ]:
class MatchingFinder:
    '''
    Implements the augmenting path algorithm to find a 1-factor or a blocking set.
    '''
    def __init__(self, graph):
        self.graph = graph
        self.n = graph.n
        # match_Y[v] stores the vertex in X matched to v in Y. -1 if unmatched.
        self.match_Y = [-1] * self.n

    def find_1_factor(self):
        '''
        Main method to find a 1-factor.
        Iterates through each vertex in X, finds an augmenting path if it's unmatched,
        and augments the matching.

        Returns:
            - (True, None) if a 1-factor is found.
            - (False, blocking_set_size) if no 1-factor exists.
        '''
        matching_size = 0
        for u_start in range(self.n):
            # Use breadth-first search (BFS) to find an augmenting path from u_start
            # parent_map stores the path taken to reach a vertex
            parent_map = {u_start: None}
            queue = deque([u_start])

            # reachable_X is the set S used to identify the blocking set
            reachable_X = {u_start}
            path_found = False

            while queue:
                u = queue.popleft() # Current vertex in X

                # Explore edges (u,v) not in the matching
                for v in self.graph.get_neighbors(u):
                    if v not in parent_map:
                        parent_map[v] = u

                        # Is v unmatched? If so, we found an augmenting path.
                        if self.match_Y[v] == -1:
                            self._augment_path(v, parent_map)
                            matching_size += 1
                            path_found = True
                            break

                        # If v is matched, continue the alternating path
                        else:
                            matched_u = self.match_Y[v]
                            if matched_u not in parent_map:
                                parent_map[matched_u] = v
                                queue.append(matched_u)
                                reachable_X.add(matched_u)
                if path_found:
                    break

            # If after the BFS, no path was found for u_start, no 1-factor exists
            if not path_found:
                return False, len(reachable_X)

        return True, None

    def _augment_path(self, v, parent_map):
        '''
        Reconstructs the augmenting path and "flips" the edges to increase
        the matching size by one.
        '''
        curr_v = v
        while curr_v is not None:
            prev_u = parent_map[curr_v]
            prev_v = parent_map.get(prev_u)
            # Add edge (u, v) to the matching
            self.match_Y[curr_v] = prev_u
            curr_v = prev_v

Trial division has an exponential time complexity whereas Miller-Rabin has a polynomial time complexity.

In [ ]:
def run_probability_experiment(n, p_values, title):
    '''
    Runs the experiment for varying p values and prints the results.
    '''
    print(title)
    print("-" * 80)
    print(f"{'p Value':<12} | {'1-Factor Rate':<15} | {'Failures (of 20)':<20} | {'Avg. Blocking Set Size':<20}")
    print("-" * 80)

    for p in p_values:
        successes = 0
        blocking_sets = []
        for _ in range(20):
            graph = BipartiteGraph(n, p)
            finder = MatchingFinder(graph)
            is_factor, blocking_set_size = finder.find_1_factor()
            if is_factor:
                successes += 1
            else:
                blocking_sets.append(blocking_set_size)

        rate = f"{(successes / 20 * 100):.0f}%"
        failures = 20 - successes
        avg_size = f"{np.mean(blocking_sets):.1f}" if blocking_sets else "N/A"

        print(f"{p:<12.4f} | {rate:<15} | {failures:<20} | {avg_size:<20}")
    print("\n")

n = 60
p_values_1 = np.arange(0.05, 0.36, 0.05)
run_probability_experiment(n, p_values_1, f"Results for n={n} (Constant p steps)")
ln_n_div_n = math.log(n) / n
p_values_2 = [c * ln_n_div_n for c in np.arange(0.1, 2.0, 0.3)]
run_probability_experiment(n, p_values_2, f"Results for n={n} (p = c * ln(n)/n)")

Results for n=60 (Constant p steps)
--------------------------------------------------------------------------------
p Value      | 1-Factor Rate   | Failures (of 20)     | Avg. Blocking Set Size
--------------------------------------------------------------------------------
0.0500       | 0%              | 20                   | 1.6                 
0.1000       | 15%             | 17                   | 19.6                
0.1500       | 40%             | 12                   | 23.7                
0.2000       | 35%             | 13                   | 22.5                
0.2500       | 30%             | 14                   | 22.8                
0.3000       | 60%             | 8                    | 23.4                
0.3500       | 55%             | 9                    | 23.0                


Results for n=60 (p = c * ln(n)/n)
--------------------------------------------------------------------------------
p Value      | 1-Factor Rate   | Failures (of 20)     | Avg. Block

This demonstrate the threshold phenomenon in random graph theory. For the existence of a 1-factor in a random bipartite graph $\mathcal{G}(n,n,p)$, the critical probability is $p_c \approx \log(n)/n$.

*   Below the threshold ($c < 1$): The probability of finding a $1$-factor is effectively zero as the graph is too sparse.
*   At the threshold ($c \approx 1$): We observe a sharp transition, with a high success rate in our trials.
*   Above the threshold (c > 1): The probability of a $1$-factor existing rapidly approaches $1$.

When we are well below the threshold, the blocking sets found have a small average size, particularly close to $1$ which is an isolated vertex, which is the most common reason for the non-existence of a 1-factor in sparse random graphs.

In [ ]:
def run_large_experiment(n, num_trials):
    '''
    Runs the bipartite graph process simulation for a large graph size.
    '''
    print(f"Question 6: Bipartite Graph Process Simulation (n={n})")
    print("-" * 80)
    print(f"{'Trial':<10} | {'Edges to End Isolation':<25} | {'Edges for 1-Factor':<25} | {'Difference':<15}")
    print("-" * 80)

    results = []
    for i in range(num_trials):
        graph = BipartiteGraph(n)

        possible_edges = [(u, v) for u in range(n) for v in range(n)]
        random.shuffle(possible_edges)

        m_iso = -1
        m_1f = -1

        for m, (u, v) in enumerate(possible_edges, 1):
            graph.add_edge(u, v)

            # Check for m_iso if it hasn't been found yet
            if m_iso == -1 and not graph.has_isolated_vertices():
                m_iso = m

            # Once isolation is gone, start checking for a 1-factor
            if m_iso != -1:
                finder = MatchingFinder(graph)
                is_factor, _ = finder.find_1_factor()
                if is_factor:
                    m_1f = m
                    break # End this trial

        results.append((m_iso, m_1f))
        print(f"{i+1:<10} | {m_iso:<25} | {m_1f:<25} | {m_1f - m_iso:<15}")

    # Calculate and print averages
    avg_m_iso = np.mean([res[0] for res in results])
    avg_m_1f = np.mean([res[1] for res in results])
    avg_diff = np.mean([res[1] - res[0] for res in results])
    print("-" * 80)
    print(f"{'Average':<10} | {avg_m_iso:<25.1f} | {avg_m_1f:<25.1f} | {avg_diff:<15.1f}")
    print("\n")

n6 = 40
run_large_experiment(n6, num_trials=10)

Question 6: Bipartite Graph Process Simulation (n=40)
--------------------------------------------------------------------------------
Trial      | Edges to End Isolation    | Edges for 1-Factor        | Difference     
--------------------------------------------------------------------------------
1          | 149                       | 222                       | 73             
2          | 170                       | 171                       | 1              
3          | 184                       | 413                       | 229            
4          | 136                       | 175                       | 39             
5          | 222                       | 224                       | 2              
6          | 190                       | 205                       | 15             
7          | 151                       | 162                       | 11             
8          | 167                       | 167                       | 0              
9          | 177   

For a $1$-factor to exist the total number of vertices in the graph must be even. In a bipartite graph this is automatically satisfied. Furthermore, there must never be any isolated vertices, otherwise it is impossible to form a matching that covers that vertex.

A random bipartite graph process starts with an edgeless graph. Edges are added one by one, chosen uniformly at random from the set of non-existent edges, until the graph is complete. There are two key moments:
*   The number of edges at which the graph first has no isolated vertices.
*   The number of edges at which the algorithm first finds a 1-factor.

The appearance of a 1-factor closely follows the disappearance of the last isolated vertex, given that there are $40^2 = 1600$ possible edges. There are no instances where the graph has a minimum degree of $1$ for a long time before a $14-factor emerges.

This suggests that for a random bipartite graph of this size, the primary obstruction to forming a $1$-factor is the existence of isolated vertices. Once this local obstruction is removed, the graph is typically complex enough to satisfy the more global condition of Hall's theorem, and a $1$-factor forms quickly.

The threshold for the disappearance of isolated vertices in a random graph $\mathcal{G}(n,p)$ is known to be at $p \approx \log(n) / n$.
If $p \ll \log(n)/n$, then the graph is very likely to have isolated vertices. If $p \gg \log(n)/n$, then the graph is very unlikely to have isolated vertices.